# AccelerateAI: Ordinal Logistic Regression

**Dataset:**
* We will use a stata data file to load from the UCLA website. This is a simulated dataset to help understand various aspects of Ordinal Logistic Regression. This notebook is inspired by https://stats.idre.ucla.edu/r/dae/ordinal-logistic-regression/ which is a R notebook from UCLA.
    * Objective: Probability for undergraduate students to apply to graduate school.
    * Exog Variables:
        * gpa – Grade Point Average, 0-4
        * pared – Flag on Parent Education: whether at least one parent went to graduate school, 0 or 1
        * public – Flag which indicates student’s undergraduate institution is public or private: 1=public and 0=private
    * Endog Variable:
        * apply – ordered categories (unlikely, somewhat likely, very likely)

**Coverage:**
* Ordinal Probit Regression
* Ordinal Logit Regression


## Ordinal Regression

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from statsmodels.miscmodels.ordinal_model import OrderedModel

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = "https://stats.idre.ucla.edu/stat/data/ologit.dta"
df = pd.read_stata(url)

In [3]:
df.head(5)

,apply,pared,public,gpa
0,very likely,0,0,3.26
1,somewhat likely,1,0,3.21
2,unlikely,1,1,3.94
3,somewhat likely,0,0,2.81
4,somewhat likely,0,0,2.53


In [4]:
df.shape

(400, 4)

In [5]:
df['apply'].dtype

CategoricalDtype(categories=['unlikely', 'somewhat likely', 'very likely'], ordered=True)

In [6]:
df['apply'].value_counts()

unlikely           220
somewhat likely    140
very likely         40
Name: apply, dtype: int64

## 1. Ordinal Probit Regression

In [7]:
model_OrdinalProbit = OrderedModel(df['apply'],
                        df[['pared', 'public', 'gpa']],
                        distr='probit')                    # Distribution instance

result_OrdinalProbit = model_OrdinalProbit.fit(method='bfgs')                     # Broyden-Fletcher-Goldfarb-Shanno (BFGS)

Optimization terminated successfully.
         Current function value: 0.896869
         Iterations: 17
         Function evaluations: 21
         Gradient evaluations: 21


In [8]:
print(result_OrdinalProbit.summary())

                             OrderedModel Results                             
Dep. Variable:                  apply   Log-Likelihood:                -358.75
Model:                   OrderedModel   AIC:                             727.5
Method:            Maximum Likelihood   BIC:                             747.5
Date:                Sat, 17 Sep 2022                                         
Time:                        11:53:12                                         
No. Observations:                 400                                         
Df Residuals:                     395                                         
Df Model:                           5                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
pared                           0.5981      0.158      3.789      0.000       0.289       0.908
p

* We have 3 coefficients that need to be estimated
    * Those 3 estimations and their standard errors can be retrieved in the summary table
* Since there are 3 categories in the target variable(unlikely, somewhat likely, very likely), we have two thresholds to estimate.
    * The first estimated threshold is the actual value
    * All the other thresholds are in terms of cumulative exponentiated increments
    * Actual threshold can be computed with the transform_threshold_params method as well which is pre-built

In [9]:
# Actual Threshold values
num_of_thresholds = 2
model_OrdinalProbit.transform_threshold_params(result_OrdinalProbit.params[num_of_thresholds:])

array([      -inf, 1.2968454 , 2.50285885,        inf])

In [10]:
# Exponentiated Increment can be added from 2nd threshold value onwards
1.2968 + np.exp(0.1873)

2.502789027494093

### Interpretation of result - Scenario1

In [11]:
# We can verify for a case where a person with GPA 3.3 and whose parents do not have an advanced degree i.e. pared=0
# and who went to a private school i.e. public = 0 would have a y* or latent variable y which can be computed as below

df.loc[(df['gpa'] == 3.3) & (df['pared'] == 1) & (df['public'] == 0)]

,apply,pared,public,gpa
249,unlikely,1,0,3.3


```y* = 0.5981 * pared + 0.0102 * public + 0.3582 * gpa
      = 0.5981 * 0 + 0.0102 * 0 + 0.3582 * 3.3
      = 0 + 0 + 1.18
      = 1.18```

Our thresholds / tau-cuts are as follows:
- Unlikely - upto 1.29
- Somewhat likely - 1.29 to 2.50
- Very likely - Greater than 2.50

Since our latent variable y is approximately 1.18, so it falls into the "Unlikely" bracket.

### Interpretation of result - Scenario2

In [12]:
# We can verify for a case where a person with GPA 3.21 and whose parents do not have an advanced degree i.e. pared=1
# and who went to a private school i.e. public = 0 would have a y* or latent variable y which can be computed as below

df.loc[(df['gpa'] == 3.21) & (df['pared'] == 1)]

,apply,pared,public,gpa
1,somewhat likely,1,0,3.21
267,very likely,1,0,3.21


```y* = 0.5981 * pared + 0.0102 * public + 0.3582 * gpa
      = 0.5981 * 1 + 0.0102 * 0 + 0.3582 * 3.21
      = 0.5981 + 0 + 1.1498
      = 1.75```

Our thresholds / tau-cuts are as follows:
- Unlikely - upto 1.29
- Somewhat likely - 1.29 to 2.50
- Very likely - Greater than 2.50

Since our latent variable y is approximately 1.75, so it falls into the "Somewhat Likely" bracket.

## 2. Ordinal Logit Regression

In [13]:
model_OrdinalLogit = OrderedModel(df['apply'],
                        df[['pared', 'public', 'gpa']],
                        distr='logit')                    # Distribution instance

result_OrdinalLogit = model_OrdinalLogit.fit(method='bfgs', disp=False)          # Broyden-Fletcher-Goldfarb-Shanno (BFGS)
print(result_OrdinalLogit.summary())

                             OrderedModel Results                             
Dep. Variable:                  apply   Log-Likelihood:                -358.51
Model:                   OrderedModel   AIC:                             727.0
Method:            Maximum Likelihood   BIC:                             747.0
Date:                Sat, 17 Sep 2022                                         
Time:                        11:53:12                                         
No. Observations:                 400                                         
Df Residuals:                     395                                         
Df Model:                           5                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
pared                           1.0476      0.266      3.942      0.000       0.527       1.569
p

* Again we have 3 coefficients to be estimated here as well
    - pared
    - public
    - gpa
* 2 intercepts for 3 categories that we have in outcome feature
    - First one is the actual threshold value for ```unlikely/somewhat likely```
    - Second one is the exponentiated increment which we can compute for ```somewhat likely/very likely```

In [14]:
# Actual Threshold values
num_of_thresholds = 2
model_OrdinalLogit.transform_threshold_params(result_OrdinalLogit.params[num_of_thresholds:])

array([      -inf, 2.20354835, 4.2989748 ,        inf])

In [15]:
# Exponentiated Increment can be added from 2nd threshold value onwards
2.2035 + np.exp(0.7398)

4.299016369307381

### Interpretation of Result - Scenario

In [16]:
# We can verify for a case where a person with GPA 3.21 and whose parents do not have an advanced degree i.e. pared=1
# and who went to a private school i.e. public = 0 would have a y* or latent variable y which can be computed as below

df.loc[(df['gpa'] == 3.21) & (df['pared'] == 1)]

,apply,pared,public,gpa
1,somewhat likely,1,0,3.21
267,very likely,1,0,3.21


```
Logit(F-unlikely) = 2.20 – (1.05 * pared – 0.05 * public + 0.62 * gpa)
                     = 2.20 - (1.05 * 1 - 0 + 0.62 * 3.21)
                     = 2.20 - (1.05 + 1.99)
                     = 2.20 - 3.04
                     = -0.84

Odds(F-unlikely) = exp(-0.84) = 0.43

(F-unlikely) = 0.43/1.43 = 0.30
```

```
Logit(F-somewhat) = 4.30 – (1.05 * pared – 0.05 * public + 0.62 * gpa)
                  = 4.30 - (1.05 * 1 - 0 + 0.62 * 3.21)
                  = 4.30 - (1.05 + 1.99)
                  = 4.30 - 3.04
                  = 1.26
Odds(F-somewhat) = exp(1.26) = 3.52

(F-somewhat) = 3.52/4.52 = 0.78
```

Probability(Unlikely) = (F-unlikely) = 0.30 = 30%

Probability(Somewhat Likely) = (F-somewhat) - (F-unlikely) = 0.78-0.30 = 0.48 = 48%
                          
Probability(Very Likely) = 1-0.78 = 0.22 = 22%

Out of the above three, Probability(Somewhat Likely) is higher and can be considered for the record where pared=1, public=0 and gpa=3.21

## Compare - Ordinal LR Probit vs Ordinal LR Logit

We can compare and observe that the prediction probabilities are not much different for each combination for probit/logit cases.

## Predictions

In [17]:
predicted = result_OrdinalLogit.model.predict(result_OrdinalLogit.params, exog=df[['pared', 'public', 'gpa']])
predicted

array([[0.54884071, 0.35932276, 0.09183653],
       [0.30558191, 0.47594216, 0.21847593],
       [0.22938356, 0.47819057, 0.29242586],
       ...,
       [0.69380357, 0.25470075, 0.05149568],
       [0.54884071, 0.35932276, 0.09183653],
       [0.50896793, 0.38494062, 0.10609145]])

Let's check for one of the user.

In [18]:
df.loc[(df['gpa'] == 2.38) & (df['pared'] == 1) & (df['public'] == 0)]

,apply,pared,public,gpa
315,somewhat likely,1,0,2.38


In [19]:
2.20 - (1.0476 * 1 + 0.6158 * 2.38)

-0.3132039999999998

In [20]:
np.exp(-0.3132039999999998)

0.7311007528015316

In [21]:
0.7311007528015316/1.7311007528015316

0.4223328720863605

In [22]:
# 315th record
predicted[315:316]

array([[0.42318088, 0.43321808, 0.14360104]])

The first category probability almost matches close to the prediction i.e. 42.2%

In [23]:
a = 4.30 - (1.0476 * 1 + 0.6158 * 2.38)
b = np.exp(a)
c = b/(1+b)

c

0.8565340073623479

This is 85.6% which is same as combination of first two categories (Unlikely + Somewhat Likely) as it is cummulative.

Same way we can identify.

That's all.